<a href="https://colab.research.google.com/github/me-ibad/NPL-Project-Task-3/blob/main/retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faiss-cpu sentence-transformers PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [3]:
# retriever.py

import os
import pickle
from typing import List
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

class Retriever:
    """
    A document retriever using SentenceTransformers for embeddings and FAISS for vector search.
    """
    def __init__(self, model_name='all-MiniLM-L6-v2', chunk_size=300, overlap=50):
        self.model = SentenceTransformer(model_name)
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.index = None
        self.documents = []
        self.embeddings = []

    def _chunk_text(self, text: str) -> List[str]:
        words = text.split()
        chunks = []
        for i in range(0, len(words), self.chunk_size - self.overlap):
            chunk = ' '.join(words[i:i+self.chunk_size])
            chunks.append(chunk)
        return chunks

    def add_documents(self, texts: List[str]):
        """
        Add and index documents. Accepts list of strings.
        """
        all_chunks = []
        for text in texts:
            chunks = self._chunk_text(text)
            all_chunks.extend(chunks)
            self.documents.extend(chunks)

        embeddings = self.model.encode(all_chunks, convert_to_numpy=True, show_progress_bar=True)
        embeddings = normalize(embeddings)

        self.embeddings = embeddings
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)

    def query(self, query_text: str, top_k=3) -> List[str]:
        """
        Retrieve top_k relevant document chunks for the given query.
        """
        query_vec = self.model.encode([query_text], convert_to_numpy=True)
        query_vec = normalize(query_vec)
        D, I = self.index.search(query_vec, top_k)
        return [self.documents[i] for i in I[0]]

    def save(self, path: str):
        """
        Save index and document chunks.
        """
        faiss.write_index(self.index, f"{path}.index")
        with open(f"{path}.pkl", "wb") as f:
            pickle.dump(self.documents, f)

    def load(self, path: str):
        """
        Load index and documents from disk.
        """
        self.index = faiss.read_index(f"{path}.index")
        with open(f"{path}.pkl", "rb") as f:
            self.documents = pickle.load(f)


In [4]:
import os
from PyPDF2 import PdfReader

def load_text_file(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def load_pdf_file(path):
    reader = PdfReader(path)
    return "\n".join([page.extract_text() or "" for page in reader.pages])

def load_documents_from_folder(folder_path):
    docs = []
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        if fname.endswith(".txt") or fname.endswith(".md"):
            docs.append(load_text_file(fpath))
        elif fname.endswith(".pdf"):
            docs.append(load_pdf_file(fpath))
    return docs


In [6]:
# Initialize
retriever = Retriever(chunk_size=21, overlap=0)

# Load your documents from the current folder
documents = load_documents_from_folder(".")

# Add documents to the retriever
retriever.add_documents(documents)

# Query the retriever
query = "What does python use?"
results = retriever.query(query)

# Show result
print("Query:", query)
print("Top result:", results[0])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: What does python use?
Top result: is a programming language that lets you work quickly and integrate systems more effectively. It is widely used in web development,


In [ ]:
def test_retriever():
    r = Retriever()
    r.add_documents(["Python is a popular programming language."])
    result = r.query("What is Python?")
    assert "Python" in result[0]
    print("✅ Test passed")

test_retriever()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Test passed
